###IMPORT LIBRARY

In [9]:
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

##LOAD DATA

In [3]:
df =pd.read_csv ("https://raw.githubusercontent.com/rahmantaufik27/dataset/main/study/credit_dataset.csv")
df.head()

,customer_id,balance_number,balance_frequency,purchases_number,purchases_conditional,purchases_installment,purchases_cash,purchases_frequency,purchases_frequency_conditional,purchases_frequency_installment,...,purchases_cash_advance,purchases_trx,limit_credit,payment_number,payment_minimum,payment_total,tenure,tax_number,tax_cash,tax_total
0,C10001,40.900749,0.818182,95.40,0.00,95.4,0.000000,0.166667,0.000000,0.083333,...,0,2,1000.0,201.802084,139.509787,0.000000,12,NaN,NaN,NaN
1,C10002,3202.467416,0.909091,0.00,0.00,0.0,6442.945483,0.000000,0.000000,0.000000,...,4,0,7000.0,4103.032597,1072.340217,0.222222,12,NaN,NaN,NaN
2,C10003,2495.148862,1.000000,773.17,773.17,0.0,0.000000,1.000000,1.000000,0.000000,...,0,12,7500.0,622.066742,627.284787,0.000000,12,NaN,NaN,NaN
3,C10004,1666.670542,0.636364,1499.00,1499.00,0.0,205.788017,0.083333,0.083333,0.000000,...,1,1,7500.0,0.000000,NaN,0.000000,12,NaN,NaN,NaN
4,C10005,817.714335,1.000000,16.00,16.00,0.0,0.000000,0.083333,0.083333,0.000000,...,0,1,1200.0,678.334763,244.791237,0.000000,12,NaN,NaN,NaN


##Exploratory Data Analysis (EDA)


In [4]:
#informasi dataframe(jumlah data, jumlah kolom, jumlah baris, tipe data, dll)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8950 entries, 0 to 8949
Data columns (total 21 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   customer_id                      8950 non-null   object 
 1   balance_number                   8950 non-null   float64
 2   balance_frequency                8950 non-null   float64
 3   purchases_number                 8950 non-null   float64
 4   purchases_conditional            8950 non-null   float64
 5   purchases_installment            8950 non-null   float64
 6   purchases_cash                   8950 non-null   float64
 7   purchases_frequency              8950 non-null   float64
 8   purchases_frequency_conditional  8950 non-null   float64
 9   purchases_frequency_installment  8950 non-null   float64
 10  purchases_cash_frequency         8950 non-null   float64
 11  purchases_cash_advance           8950 non-null   int64  
 12  purchases_trx       

In [5]:
#jumlah missing values setiap column
df.isna().sum()

customer_id                           0
balance_number                        0
balance_frequency                     0
purchases_number                      0
purchases_conditional                 0
purchases_installment                 0
purchases_cash                        0
purchases_frequency                   0
purchases_frequency_conditional       0
purchases_frequency_installment       0
purchases_cash_frequency              0
purchases_cash_advance                0
purchases_trx                         0
limit_credit                          1
payment_number                        0
payment_minimum                     313
payment_total                         0
tenure                                0
tax_number                         8950
tax_cash                           8950
tax_total                          8950
dtype: int64

In [6]:
#informasi statistik dataframe
df.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
balance_number,8950.0,1564.474828,2081.531879,0.000000,128.281915,873.385231,2054.140036,19043.13856
balance_frequency,8950.0,0.877271,0.236904,0.000000,0.888889,1.000000,1.000000,1.00000
purchases_number,8950.0,1003.204834,2136.634782,0.000000,39.635000,361.280000,1110.130000,49039.57000
purchases_conditional,8950.0,592.437371,1659.887917,0.000000,0.000000,38.000000,577.405000,40761.25000
purchases_installment,8950.0,411.067645,904.338115,0.000000,0.000000,89.000000,468.637500,22500.00000
purchases_cash,8950.0,978.871112,2097.163877,0.000000,0.000000,0.000000,1113.821139,47137.21176
purchases_frequency,8950.0,0.490351,0.401371,0.000000,0.083333,0.500000,0.916667,1.00000
purchases_frequency_conditional,8950.0,0.202458,0.298336,0.000000,0.000000,0.083333,0.300000,1.00000
purchases_frequency_installment,8950.0,0.364437,0.397448,0.000000,0.000000,0.166667,0.750000,1.00000
purchases_cash_frequency,8950.0,0.135144,0.200121,0.000000,0.000000,0.000000,0.222222,1.50000


In [24]:
# Menampilkan 5 data teratas dari dataset
print("Data Head:")
print(data.head())

Data Head:
  customer_id  balance_number  balance_frequency  purchases_number  \
0      C10001       40.900749           0.818182             95.40   
1      C10002     3202.467416           0.909091              0.00   
2      C10003     2495.148862           1.000000            773.17   
3      C10004     1666.670542           0.636364           1499.00   
4      C10005      817.714335           1.000000             16.00   

   purchases_conditional  purchases_installment  purchases_cash  \
0                   0.00                   95.4        0.000000   
1                   0.00                    0.0     6442.945483   
2                 773.17                    0.0        0.000000   
3                1499.00                    0.0      205.788017   
4                  16.00                    0.0        0.000000   

   purchases_frequency  purchases_frequency_conditional  \
0             0.166667                         0.000000   
1             0.000000                         

##PREPROCESSING

In [27]:
# Standarisasi fitur
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

/usr/local/lib/python3.10/dist-packages/sklearn/utils/extmath.py:1047: RuntimeWarning: invalid value encountered in divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
/usr/local/lib/python3.10/dist-packages/sklearn/utils/extmath.py:1052: RuntimeWarning: invalid value encountered in divide
  T = new_sum / new_sample_count
/usr/local/lib/python3.10/dist-packages/sklearn/utils/extmath.py:1072: RuntimeWarning: invalid value encountered in divide
  new_unnormalized_variance -= correction**2 / new_sample_count


In [25]:
# Memilih fitur yang akan digunakan untuk clustering
X = data.iloc[:, 1:].values  # Mengabaikan kolom ID

In [28]:
# Inisialisasi model KMeans dengan jumlah klaster yang diinginkan
k = 3  # Jumlah klaster yang diinginkan
kmeans = KMeans(n_clusters=k, random_state=42)


##MODELING

k-MEANS

In [41]:
# Buat model K-Means
kmeans = KMeans(n_clusters=k)